# Sine Wave Test - Sensor Validation

**Purpose:** Verify AS5600 sensor tracks stepper during continuous sine wave motion.

**What matters:** Mechanical slack (stepper - sensor) should be < 0.5° for good sensor-stepper agreement.

**What doesn't matter:** Tracking error (in open-loop, we're not trying to follow a trajectory).

In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import os
import glob

# --- CONFIGURATION ---
TEST_TYPE = 'sine'

# Option 1: Manually specify file
FILE_PATH = f'../logs/{TEST_TYPE}/sine_20260112-163527.csv'

# Option 2: Auto-detect latest file (uncomment to use)
# csv_files = glob.glob(f'../logs/{TEST_TYPE}/*.csv')
# if csv_files:
#     FILE_PATH = max(csv_files, key=os.path.getmtime)
#     print(f"Auto-detected: {os.path.basename(FILE_PATH)}")
# else:
#     FILE_PATH = None

In [2]:
# Load and parse the CSV data
if not os.path.exists(FILE_PATH):
    print(f"❌ Error: File not found at {FILE_PATH}")
    print(f"Current directory: {os.getcwd()}")
else:
    # Read CSV, skipping comment lines
    df = pd.read_csv(FILE_PATH, comment='#')
    
    # Convert timestamp to seconds
    df['time_s'] = df['timestamp_ms'] / 1000.0
    
    # Calculate derived metrics
    df['mech_slack'] = df['stepper_deg'] - df['sensor_deg']
    
    print(f"✅ Loaded {len(df)} data points from: {os.path.basename(FILE_PATH)}")
    print(f"   Duration: {df['time_s'].iloc[-1] - df['time_s'].iloc[0]:.2f} seconds")
    print(f"   Sample rate: {len(df) / (df['time_s'].iloc[-1] - df['time_s'].iloc[0]):.1f} Hz")

✅ Loaded 4355 data points from: sine_20260112-163527.csv
   Duration: 21.80 seconds
   Sample rate: 199.8 Hz


In [3]:
# Open-loop validation metrics
print("=" * 60)
print("SINE WAVE TEST - SENSOR VALIDATION")
print("=" * 60)

print("\n🎯 PRIMARY METRIC (What matters):")
print(f"   Mechanical Slack (Stepper - Sensor):")
print(f"   Mean:      {df['mech_slack'].mean():+7.3f}°")
print(f"   Std Dev:   {df['mech_slack'].std():7.3f}°")
print(f"   Max:       {df['mech_slack'].abs().max():7.3f}°")

print("\n📊 VERDICT:")
if df['mech_slack'].abs().max() < 0.5:
    print("   ✅ EXCELLENT - Sensor tracks stepper accurately")
elif df['mech_slack'].abs().max() < 2.0:
    print("   ⚠️  ACCEPTABLE - Minor mechanical slack")
else:
    print("   ❌ POOR - Check belt tension, magnet, or STEPS_PER_DEG")

print("\n" + "=" * 60)

SINE WAVE TEST - SENSOR VALIDATION

🎯 PRIMARY METRIC (What matters):
   Mechanical Slack (Stepper - Sensor):
   Mean:       -0.030°
   Std Dev:     0.582°
   Max:         1.072°

📊 VERDICT:
   ⚠️  ACCEPTABLE - Minor mechanical slack



## Sensor vs Stepper Comparison

In [4]:
# Compare sensor vs stepper (ignore target in open-loop)
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    x=df['time_s'], y=df['sensor_deg'],
    name='Sensor (AS5600)', mode='lines',
    line=dict(color='rgb(50, 205, 50)', width=2)
))

fig1.add_trace(go.Scatter(
    x=df['time_s'], y=df['stepper_deg'],
    name='Stepper (Steps)', mode='lines',
    line=dict(color='rgb(255, 165, 0)', width=2, dash='dash')
))

fig1.update_layout(
    title='Sine Wave: Sensor vs Stepper',
    xaxis_title='Time (s)',
    yaxis_title='Position (deg)',
    height=500,
    hovermode='x unified',
    template='plotly_white'
)

fig1.show()

## Mechanical Slack (Error)

In [5]:
# Mechanical slack = stepper - sensor
fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    x=df['time_s'], y=df['mech_slack'],
    name='Mechanical Slack', mode='lines',
    line=dict(color='rgb(138, 43, 226)', width=2),
    fill='tozeroy', fillcolor='rgba(138, 43, 226, 0.1)'
))

fig2.add_hline(y=0, line_dash="dash", line_color="gray")
fig2.add_hrect(y0=-0.5, y1=0.5, fillcolor="green", opacity=0.1,
               annotation_text="Good (±0.5°)", annotation_position="right")

fig2.update_layout(
    title='Mechanical Slack: Stepper - Sensor',
    xaxis_title='Time (s)',
    yaxis_title='Slack (deg)',
    height=400,
    hovermode='x unified',
    template='plotly_white'
)

fig2.show()

## Error Distribution

In [6]:
# Histogram of mechanical slack
fig3 = go.Figure()

fig3.add_trace(go.Histogram(
    x=df['mech_slack'], nbinsx=50,
    marker_color='rgb(138, 43, 226)',
    opacity=0.7
))

fig3.update_layout(
    title='Mechanical Slack Distribution (Should be narrow and centered)',
    xaxis_title='Slack (deg)',
    yaxis_title='Count',
    height=400,
    template='plotly_white'
)

fig3.show()

---
## Interpretation

**Good sensor:** Mechanical slack < 0.5° (tight coupling between stepper and sensor)

**If slack is large (>2°):**
- Belt slipping
- Loose magnet
- Wrong `STEPS_PER_DEG` in firmware
- Backlash in coupling

**Once validated here, you can use this sensor for closed-loop control.**